In [33]:
import os.path
import pandas as pd
import json
from nested_lookup import nested_lookup as nl

In [9]:
file_path = os.path.join('data','2018-06-08-0000Z.json')
with open(file_path, 'r', encoding='utf-8') as fp:
    file = json.load(fp)

In [94]:
for key in file.keys():
    print(f'{key:9} | {str(type(file[key])):15} | {file[key] if not key == "acList" else None}')
print('Total:',len(file.keys()))

src       | <class 'int'>   | 1
feeds     | <class 'list'>  | [{'id': 1, 'name': 'From Consolidator', 'polarPlot': False}]
srcFeed   | <class 'int'>   | 1
showSil   | <class 'bool'>  | True
showFlg   | <class 'bool'>  | True
showPic   | <class 'bool'>  | True
flgH      | <class 'int'>   | 20
flgW      | <class 'int'>   | 85
acList    | <class 'list'>  | None
totalAc   | <class 'int'>   | 9999
lastDv    | <class 'str'>   | 9999
shtTrlSec | <class 'int'>   | 65
stm       | <class 'int'>   | 9999999999999
Total: 13


In [122]:
ac_key_list = []
for ac in file['acList']:
    ac_key_list.extend(ac.keys())
ac_key_list = sorted(set(ac_key_list))
# for key in ac_key_list:
#     print(key)
print('\t'.join([key for key in ac_key_list]))
print('Total:',len(ac_key_list))

Alt	AltT	Bad	CMsgs	CNum	Call	CallSus	Cos	Cou	EngMount	EngType	Engines	FSeen	FlightsCount	From	GAlt	Gnd	HasPic	HasSig	Help	Icao	Id	InHg	Interested	Lat	Long	Man	Mdl	Mil	Mlat	Op	OpIcao	PicX	PicY	PosStale	PosTime	Rcvr	Reg	ResetTrail	Sat	Sig	Spd	SpdTyp	Species	Sqk	Stops	TAlt	TSecs	TT	TTrk	Tag	Tisb	To	Trak	TrkH	Trt	Type	Vsi	VsiT	WTC	Year
Total: 61


In [121]:
printList = []
for item in nl('Call',file['acList']):
    printList.append(item)
print('\t'.join(printList))

AAL160	N314GT	N1487T	ANR5022	KAL504	ELY398	VLG83TJ	FDY1061	N716AS	LNI929	N62Q	AAL1634	SNJ92	N540M	AFR441	N851EF	DAL303	SWA763	SKY513	N551GT	N4402T	DAL547	N340Q	N204TF	SKW5927	SKY705	CXA8473	JST259	TAP1865	GIA129	N1JS	WJA1755	143	N4402N	N177BD	AAL148	BAW15	N457JD	AAL110	N733WT	N71551	UAL990	N352VA	TGW280	JAI233	HKE850	THY114	BEL205	AAL704	N9522S	AAL899	N850EP	UAL2015	N524CW	CPA505	OAE601	THY64J	EVA698	AAL574	N551XJ	CAP900	N4404F	N421SX	ADO15	APJ103	CXI4918	BAW176A	72055	CAP3263	N91666	AAL1506	AAL1588	HVN579	N164BL	ANA792	TVS4101	UAL619	JBU357	DAL1628	N464TT	N	AAL1790	ASH6329	KAP13	TCX355	ABW476	N898DD	SKW3086	SNJ72	OMA102	AMX437	AAL1216	SRZ4171	N12ZZ	NRD	JZR1541	ASL36P	AFR6731	SWA147	RLK203	FDX320	1632	AM242	KAR1726	AAL849	JBU1623	DAL2596	QFA840	N988DV	N9454X	EXER 04	N881JC	00000000	N718GH	UAL70	FDX597	N33142	A8C825	1881	TVS1240	ROOK43	LNI571	AMF0562	AAL158	CFMPA	ASA1088	RPA3479	GIA103	JIA5654	NKS619	TEST1234	RYR8731	JIA5207	UAL1730	UKP22	VLG3312	SWA1497	VOI120	N7063M	DAL2186	MDA779	WJA

In [30]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

import numpy as np

x = []
y = []
z = flight['Altitude'][20:70]

for pos in flight['Position'][20:70]:
    xpos,ypos = pos.split(sep=',')
    x.append(xpos)
    y.append(ypos)

interval = flight['Timestamp'][20]-flight['Timestamp'][37]

time = flight['UTC'][20:70]
relVel50 = [vel / 10 for vel in flight['Speed']]

trace = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    text=time,
    mode='lines+markers',
    line=dict(width=5),
    marker=dict(
        size=relVel50,
        color=relVel50,
        colorscale=[[0, 'rgb(0,0,255)'], [1, 'rgb(255,0,0)']]),
    name='Trace 1')
data = [trace]

layout = go.Layout(
    title='Antonov An-148 RA-61704 Saratov Airlines [Taxing and until {}secs after Take-off]'.format(interval),
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(0, 0, 0)',
            zerolinecolor='rgb(0, 0, 0)',
            showbackground=False,
            backgroundcolor='rgb(230, 230, 230)'),
        yaxis=dict(
            gridcolor='rgb(0, 0, 0)',
            zerolinecolor='rgb(0, 0, 0)',
            showbackground=False,
            backgroundcolor='rgb(230, 230, 230)'),
        zaxis=dict(
            gridcolor='rgb(0, 0, 0)',
            zerolinecolor='rgb(0, 0, 0)',
            showbackground=False,
            backgroundcolor='rgb(230, 230, 230)'),
        camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.5, y=0.5, z=0.3)),
        dragmode="orbit"))
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='jupyter-parametric_plot')